In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

In [3]:
graphPath = "/data/graphDFSample"

In [31]:
from pyspark.sql.functions import explode, collect_list, size, col, row_number, sort_array, udf
from pyspark.sql.types import *
from pyspark.sql import Window

In [81]:
users = sparkSession.read.parquet(graphPath) \
                    .withColumn("friend", explode('friends'))\
                    .groupBy("friend").agg(collect_list("user").alias("users"))

In [25]:
users_sorted = users.select('friend',sort_array('users').alias('sorted'))\
                    .filter(size('sorted')>1)

In [27]:
def explode_x(l):
    result = []
    for i in range(0,len(l)):
        for j in range(i+1,len(l)):
            result.append([l[i],l[j]])
    return result

In [40]:
udf_explode = udf(explode_x, ArrayType(StructType([StructField('1',
                                                               IntegerType(),
                                                               False),
                                                   StructField('2',
                                                               IntegerType(),
                                                               False)])))

In [43]:
users_exploded = users_sorted.select('friend', udf_explode('sorted').alias('pairs_list')) \
                             .select('friend',explode('pairs_list').alias('pairs'))

In [45]:
users_pairs = users_exploded.groupBy('pairs').count()

In [47]:
window = Window.orderBy(col("count").desc())

In [83]:
top49 = users_pairs.withColumn("row_num", row_number().over(window)) \
                    .filter("row_num<50") \
                    .select('pairs', 'count')

In [ ]:
output = top49.collect()

In [80]:
for i in output:
    print '{0} {1} {2}'.format(i[1],i.pairs[0],i.pairs[1])

3206 27967558 42973992
3130 20158643 42973992
3066 22582764 42973992
3044 21864412 51640390
3021 17139850 51640390
3010 14985079 51640390
2970 17139850 21864412
2913 20158643 27967558
2903 22280814 51151280
2870 23848749 51640390
2855 20158643 22582764
2849 20158643 44996025
2846 22280814 42973992
2784 21864412 23848749
2779 31964081 51640390
2776 39205988 51640390
2754 17139850 23848749
2749 22582764 27967558
2728 50561859 51640390
2724 15485897 51640390
2700 28135661 42973992
2655 22280814 27967558
2653 42973992 43548989
2639 26755857 51640390
2621 14635589 51640390
2608 15485897 17139850
2606 17139850 26755857
2601 21864412 39205988
2600 8406745 51640390
2599 37735419 51640390
2597 20158643 28135661
2585 21864412 31964081
2585 40003405 42973992
2581 27967558 43548989
2579 23848749 31964081
2578 15485897 21864412
2578 27967558 28135661
2577 42973992 64755069
2574 51151280 57077210
2573 20158643 43548989
2566 21864412 26755857
2564 22280814 64755069
2561 42973992 44996025
2556 1713985